# AAL Master Papermill
Use papermill to create all notebooks and outputs of the AAL calculator
## Workflow:
1. **Run notebook:** `Prep_AAL_Inputs.ipynb`
2. **Run notebook:** `AAL-Calculator.ipynb`
3. **Run notebook:** `CreateAALResults.ipynb`
4. **Run notebook:** `AAL-ResultSummary.ipynb`
5. **Run notebook:** `AAL-AnomalyReport.ipynb`

In [ ]:
from time import time
import papermill as pm
from glob import glob
import pathlib as pl
import sys; sys.path.append('../core')
from risk_refactor import *

### User inputs

In [ ]:
project = 'DC'
models = ['F02']
books = ['MB', 'Uncorrelated', 'Uniform']
tri = ''  # if IT IS NOT a TRI run
# tri = '_TRI'  # if IT IS a TRI run
output_dir_name = 'outputs'

### Manage and organize file structure

In [ ]:
output_dir = join(os.getcwd(), output_dir_name, project + tri)
dir_json = setup_file_structure(output_dir, models)
notebook_dir = join(output_dir, 'notebooks')
if not exists(notebook_dir): os.makedirs(notebook_dir)
print('Output location:', output_dir)

## **Run notebook:** `Prep_AAL_Inputs.ipynb`

In [ ]:
st = time()

for mod in models:
    print(mod)
    paramdict = {'project': project,
                 'model': mod,
                 'TRI': tri,
                 'wse_dir': dir_json[mod][WSE],
                 'weights_dir': dir_json[mod][WEIGHTS]}
    pm.execute_notebook('Prep_AAL_inputs.ipynb', join(notebook_dir, '_'.join([project, mod, 'Prep_AAL_inputs.ipynb'])), parameters=paramdict, kernel_name='python3')
    
print(round((time()-st)/60, 2), 'minutes to run')

In [ ]:
# Upload to s3
for mod in models:
    print(mod)
    outwses = glob(join(dir_json[mod][WSE], '*.csv'))
    outwts = glob(join(dir_json[mod][WEIGHTS], '*.csv'))
    if tri:
        for wse in outwses:
            book = os.path.basename(wse).split('_')[-2]
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/TRI/{2}/WSE_{1}_{2}_{3}_TRI.csv'.format(wse, project, mod, book))
        for wt in outwts:
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/TRI/{2}/{1}_{2}_weights_TRI.csv'.format(wt, project, mod))
    else:
        for wse in outwses:
            book = os.path.basename(wse).split('_')[-1].split('.')[0]
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/{2}/WSE_{1}_{2}_{3}.csv'.format(wse, project, mod, book))
        for wt in outwts:
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/{2}/{1}_{2}_weights.csv'.format(wt, project, mod))

## **Run notebook:** `AAL-Calculator.ipynb`

### Parameters

In [ ]:
# The Hazus data:
hzDDFnIdJson = r'hazusdepthdmgfns\Building_DDF_Full_LUT_Hazus3p0.json'

# Damage categories:
catgroups = {'singFam_1Story_NoBasement' : [105, 129, 132, 139, 173],
             'singFam_2Story_NoBasement' : [107, 130, 136, 140, 174],
             'singFam_3Story_NoBasement' : [109],
             'singFam_1Story_Basement' : [106, 121, 133, 181, 704],
             'singFam_2Story_Basement' : [108, 122, 137],
             'singFam_3Story_Basement' : [110, 123],
             'mobileHome': [189, 191, 192, 203]}

field_map = {'Unique Building ID' : 'plus_code',
             'Damage Code': 'damage_code',
             'Building Limit' : 'bldg_limit',
             'Building Deduction' : 'bldg_ded',
             'Content Limit' : 'cnt_limit',
             'Content Deduction': 'cnt_ded',
             'Ground Elevation': 'GroundElev',
             'First Floor Height': 'first_floor_elev'}

### Run notebook

In [ ]:
st = time()

root = pl.Path(os.getcwd())
for mod in models:
    for book in books:
        print(mod, book)
        wse_path = [x for x in glob(join(dir_json[mod][WSE], '*.csv')) if book in x][0]
        if mod[0] == 'P':
            weights_file = glob(join(dir_json[mod][WEIGHTS], '*.csv'))[0]
        else:
            weights_file = ''
        
        scenname = pl.Path(wse_path).stem.replace('WSE_', '')
        paramdict = dict(root_dir = str(root),
                         wse_file = wse_path,
                         weights_path = weights_file,
                         curve_groups = catgroups,
                         structure_cols = field_map,
                         out_AAL = join(dir_json[mod][AAL], 'AAL_{}.csv'.format(scenname)),
                         out_loss = join(dir_json[mod][AAL], 'Losses_{}.csv'.format(scenname)),
                         scen = scenname)
    
        print('Using weights file:', weights_file)
        out_nb = join(dir_json[mod][AAL], 'tool_{}.ipynb'.format(scenname))
        pm.execute_notebook('AAL-Calculator.ipynb', out_nb, parameters=paramdict, kernel_name='python3')
        os.system('jupyter nbconvert {}'.format(out_nb))
        
print(round((time()-st)/60, 2), 'minutes to run')

In [ ]:
# Upload to s3
for mod in models:
    print(mod)
    outfiles = [x for x in glob(join(dir_json[mod][AAL], '*')) if not 'Losses' in x]
    if tri:
        for f in outfiles:
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/{2}/{3}'.format(f, project, mod, os.path.basename(f).replace('_TRI', '')))
    else:
        for f in outfiles:
            os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/{2}/{3}'.format(f, project, mod, os.path.basename(f)))

## **Run notebook:** `CreateAALResults.ipynb`

### Run notebook

In [ ]:
st = time()

for book in books:
    paramdict = {'project': project,
                 'book': book,
                 'TRI': tri,
                 'output_dir': dir_json[FAAL]}
    
    print('Working on {} for {}'.format(book, project))
    pm.execute_notebook('CreateAALResults.ipynb', join(notebook_dir, '_'.join([project, book, 'CreateAALResults.ipynb'])), parameters=paramdict, kernel_name='python3')
    
print(round((time()-st)/60, 2), 'minutes to run')

In [ ]:
# Upload to s3
outfiles = glob(join(dir_json[FAAL], '*'))
if tri:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/TRI/AALs/{2}'.format(f, project, os.path.basename(f)))
else:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/AALs/{2}'.format(f, project, os.path.basename(f)))

## **Run notebook:** `AAL-ResultSummary.ipynb`

### Run notebook

In [ ]:
st = time()

for book in books:
    paramdict = {'project': project,
                 'book': book,
                 'TRI': tri,
                 'outlier_threshold': 3000}
    
    print('Working on {} for {}'.format(book, project))
    out_nb = join(dir_json[SUM], '_'.join([project, book, 'AAL-ResultSummary.ipynb']))
    pm.execute_notebook('AAL-ResultSummary.ipynb', out_nb, parameters=paramdict, kernel_name='python3')
    os.system('jupyter nbconvert {}'.format(out_nb))
    
print(round((time()-st)/60, 2), 'minutes to run')

In [ ]:
# Upload to s3
outfiles = glob(join(dir_json[SUM], '*.html'))
if tri:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/TRI/AALs/{2}'.format(f, project, os.path.basename(f)))
else:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/AALs/{2}'.format(f, project, os.path.basename(f)))

## **Run notebook:** `AAL-AnomalyReport.ipynb`

### Run notebook

In [ ]:
st = time()

for book in books:
    paramdict = {'project': project,
                 'book': book,
                 'TRI': tri,
                 'anom_threshold': 3000}
    
    print('Working on {} for {}'.format(book, project))
    out_nb = join(dir_json[ANOM], '_'.join([project, book, 'AAL-AnomalyReport.ipynb']))
    pm.execute_notebook('AAL-AnomalyReport.ipynb', out_nb, parameters=paramdict, kernel_name='python3')
    os.system('jupyter nbconvert {}'.format(out_nb))
    
print(round((time()-st)/60, 2), 'minutes to run')

In [ ]:
# Upload to s3
outfiles = glob(join(dir_json[ANOM], '*.html'))
if tri:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/TRI/AALs/{2}'.format(f, project, os.path.basename(f)))
else:
    for f in outfiles:
        os.system('aws s3 cp {0} s3://pfra/RiskAssessment/{1}/Results/AALs/{2}'.format(f, project, os.path.basename(f)))

# END